In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import logging
import requests
import datetime as dt
import pylab

import scipy.stats as stats
from matplotlib import pyplot

import numpy as np

import warnings
warnings.filterwarnings('ignore')

logging.getLogger().setLevel(logging.INFO)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os, sys

home = os.environ["HOME"]
sys.path.append(f"{home}/Desktop/project-ursa/lib")

import imp

import monte_carlo
imp.reload(monte_carlo)

import data_utils
imp.reload(data_utils)

import data_analysis
imp.reload(data_analysis)

import mining_constants
imp.reload(mining_constants)

import machine_npv
imp.reload(machine_npv)

<module 'machine_npv' from '/Users/merkle-employee/Desktop/project-ursa/lib/machine_npv.py'>

In [5]:
machine_params   = mining_constants.machine_params()
machine_duration = 1.2
ky = "S19J"
params = {
    "asic_energy_consumption" : machine_params[ky]["energy_consumption"],
    "asic_hash_rate"          : machine_params[ky]["hash_rate"],
    "electricity_cost"        : 0.05,
    "machine_duration"        : machine_duration,
    "sample_rate"             : mining_constants.SAMPLE_RATE_WEEKLY,
    "mc_size"                 : 1000,
    "interest_rate"           : 0.05,
    "asic_number"             : 1,
    "asic_efficiency"         : 1.0,
    "pool_fee"                : 0.005,
    "analysis_denomination"   : mining_constants.ASSET_USD
}

##
du = data_utils.DataUtility("")
du._raw_network_price_df    = pd.read_csv("../../data/raw_network_price.csv").drop("Unnamed: 0", axis="columns")

du._raw_network_price_df.time = pd.to_datetime(du._raw_network_price_df.time)
du._df_feature() 
du._aggregate_weekly_df() 

machine_prices = data_utils.machine_market_prices("../../data/machine_prices.csv")
du._weekly_machine_price_df = du._join_machine_price(machine_prices, HR_ROLLING_AVG=30)

weekly_df = du._weekly_network_price_df
machine_df = du._weekly_machine_price_df

## 
sp = monte_carlo.GBMPriceLinearHashRate()
sp.fit(weekly_df, sample_rate=mining_constants.SAMPLE_RATE_WEEKLY)

## 
k = len(weekly_df) - 1

# start_time = weekly_df.reset_index().loc[k, "time"]
# hash_rate  = weekly_df.reset_index().loc[k, "hashrate"]
# hash_index = weekly_df.reset_index().loc[k, "hash_index"]
# btc_close  = weekly_df.reset_index().loc[k, "close"]

m_usd_bsm = machine_npv.MachineNPV_USD_BSM(sp, params)
# usd_bsm   = m_usd_bsm.price(start_time, btc_close, hash_rate)
import implied_payback_time

ipt    = implied_payback_time.ImpliedPayoffTime(params, m_usd_bsm, machine_df.tail(5))
ipt_df = ipt.compute(machine_type=mining_constants.MACHINE_M20, alpha=0.75, beta=0.5, MAX_DURATION=5)


INFO:root:precomputing option_df: k 0 time 2022-08-28 hash-rate 209709319.55404043 btc 19554.08
INFO:root:precomputing option_df: k 1 time 2022-09-04 hash-rate 215699327.77267936 btc 20003.46
INFO:root:precomputing option_df: k 2 time 2022-09-11 hash-rate 220803155.7704675 btc 21836.45
INFO:root:precomputing option_df: k 3 time 2022-09-18 hash-rate 227550574.75238138 btc 19417.72
INFO:root:precomputing option_df: k 4 time 2022-09-25 hash-rate 227996733.6188044 btc 18809.94


In [6]:
# import data_utils

# du = data_utils.DataUtility("")
# du._raw_network_price_df    = pd.read_csv("../../data/raw_network_price.csv").drop("Unnamed: 0", axis="columns")

# du._raw_network_price_df.time = pd.to_datetime(du._raw_network_price_df.time)
# du._df_feature() 
# du._aggregate_weekly_df() 

# machine_prices = data_utils.machine_market_prices("../../data/machine_prices.csv")
# du._weekly_machine_price_df = du._join_machine_price(machine_prices, HR_ROLLING_AVG=30)

# weekly_df = du._weekly_network_price_df
# machine_df = du._weekly_machine_price_df

# import monte_carlo
# imp.reload(monte_carlo)
# 
# sp = monte_carlo.GBMPriceLinearHashRate()
# sp.fit(
#     weekly_df, 
#     sample_rate = mining_constants.SAMPLE_RATE_WEEKLY
# )

In [8]:
# ## 
# k = len(weekly_df) - 1

# start_time = weekly_df.reset_index().loc[k, "time"]
# hash_rate  = weekly_df.reset_index().loc[k, "hashrate"]
# hash_index = weekly_df.reset_index().loc[k, "hash_index"]
# btc_close  = weekly_df.reset_index().loc[k, "close"]

# print("week: ", start_time, "hashrate: ", hash_rate, "hash-index: ", hash_index, "btc-close: ", btc_close)

# machine_params   = mining_constants.machine_params()
# machine_duration = 1.2
# ky = "S19J"
# params = {
#     "asic_energy_consumption" : machine_params[ky]["energy_consumption"],
#     "asic_hash_rate"          : machine_params[ky]["hash_rate"],
#     "electricity_cost"        : 0.03,
#     "machine_duration"        : machine_duration,
#     "sample_rate"             : mining_constants.SAMPLE_RATE_WEEKLY,
#     "mc_size"                 : 1000,
#     "interest_rate"           : 0.05,
#     "asic_number"             : 1,
#     "asic_efficiency"         : 1.0,
#     "pool_fee"                : 0.005,
#     "analysis_denomination"   : mining_constants.ASSET_USD
# }

In [9]:
# import machine_npv
# imp.reload(machine_npv)
# 
# m_usd_bsm = machine_npv.MachineNPV_USD_BSM(sp, params)
# usd_bsm   = m_usd_bsm.price(start_time, btc_close, hash_rate)
# 
# ipt    = implied_payback_time.ImpliedPayoffTime(params, m_usd_bsm, machine_df.tail(10))
# ipt_df = ipt.compute(machine_type=mining_constants.MACHINE_M20, alpha=0.75, beta=0.5, MAX_DURATION=5)
# 
# e = ipt_df.ipb_ttm.values - [0.61776062, 0.54054054, 0.54054054, 0.5019305 , 0.61776062, 0.73359073, 0.73359073, 0.65637066, 0.83011583, 0.88803089]
# assert round(sum(abs(e)), 3) == 0

In [12]:
# ipt_df.ipb_ttm.values

In [ ]:
# plt.figure(figsize=(10, 4))
# plt.plot(ipt_df.ipb_npv)
# plt.plot(ipt_df.eff_npv)
# plt.grid()

In [ ]:
# plt.plot(ipt_df.ipb_ttm)

In [ ]:
# ipt_df.ipb_ttm.values

In [ ]:
# when 1 plus 1 equaled 3
# and money did seem to grow on a tree
# reckless actions of few
# cursed a whole lot - who knew
# shit i didnt expect them to fire me